In [6]:
%load_ext autoreload
%autoreload 2

In [7]:
import fugashi

text = """
日本には４つの大島がある。
""".strip()

In [8]:
tagger = fugashi.Tagger()

In [9]:
print(tagger(text))

[日本, に, は, ４, つ, の, 大島, が, ある, 。]


In [10]:
tokens = tagger(text)
tokens[0]

日本

In [11]:
tab = "\t"
for token in tokens:
    print(f"{token}\t{str.join(tab, token.feature_raw.split(','))}")

日本	名詞	固有名詞	地名	国	*	*	ニッポン	日本	日本	ニッポン	日本	ニッポン	固	*	*	*	*	*	*	国	ニッポン	ニッポン	ニッポン	ニッポン	3	*	*	7821659499274752	28455
に	助詞	格助詞	*	*	*	*	ニ	に	に	ニ	に	ニ	和	*	*	*	*	*	*	格助	ニ	ニ	ニ	ニ	*	名詞%F1	*	7745518285496832	28178
は	助詞	係助詞	*	*	*	*	ハ	は	は	ワ	は	ワ	和	*	*	*	*	*	*	係助	ハ	ハ	ハ	ハ	*	"動詞%F2@0	名詞%F1	形容詞%F2@-1"	*	8059703733133824	29321
４	名詞	数詞	*	*	*	*
つ	接尾辞	名詞的	助数詞	*	*	*	ツ	つ	つ	ツ	つ	ツ	和	*	*	*	*	*	B1WS2WB3D4D5WB6D7WB8D9WB	助数	ツ	ツ	ツ	ツ	*	C3	*	6685589076320768	24322
の	助詞	格助詞	*	*	*	*	ノ	の	の	ノ	の	ノ	和	*	*	*	*	*	*	格助	ノ	ノ	ノ	ノ	*	名詞%F1	*	7968444268028416	28989
大島	名詞	普通名詞	一般	*	*	*	オオシマ	大島	大島	オーシマ	大島	オーシマ	和	*	*	*	*	*	*	体	オオシマ	オオシマ	オオシマ	オオシマ	1	C1	*	1262522850222592	4593
が	助詞	格助詞	*	*	*	*	ガ	が	が	ガ	が	ガ	和	*	*	*	*	*	*	格助	ガ	ガ	ガ	ガ	*	"動詞%F2@0	名詞%F1"	*	2168520431510016	7889
ある	動詞	非自立可能	*	*	五段-ラ行	終止形-一般	アル	有る	ある	アル	ある	アル	和	*	*	*	*	*	*	用	アル	アル	アル	アル	1	C3	*	334260158472875	1216
。	補助記号	句点	*	*	*	*	*	。	。	*	。	*	記号	*	*	*	*	*	*	補助	*	*	*	*	*	*	*	6880571302400	25


In [12]:
import pandas as pd
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 50
token.feature._asdict()

{'pos1': '補助記号',
 'pos2': '句点',
 'pos3': '*',
 'pos4': '*',
 'cType': '*',
 'cForm': '*',
 'lForm': '*',
 'lemma': '。',
 'orth': '。',
 'pron': '*',
 'orthBase': '。',
 'pronBase': '*',
 'goshu': '記号',
 'iType': '*',
 'iForm': '*',
 'fType': '*',
 'fForm': '*',
 'iConType': '*',
 'fConType': '*',
 'type': '補助',
 'kana': '*',
 'kanaBase': '*',
 'form': '*',
 'formBase': '*',
 'aType': '*',
 'aConType': '*',
 'aModType': '*',
 'lid': '6880571302400',
 'lemma_id': '25'}

In [13]:
data = pd.concat([pd.DataFrame.from_dict({"word": str(token), **token.feature._asdict()}, orient="index").T 
                  for token in tagger(text)]).reset_index(drop=True)

attr = ["orth", "lemma", "pos1", "pos2", "cType", "kana"]

data[attr]

,orth,lemma,pos1,pos2,cType,kana
0,日本,日本,名詞,固有名詞,*,ニッポン
1,に,に,助詞,格助詞,*,ニ
2,は,は,助詞,係助詞,*,ハ
3,None,None,名詞,数詞,*,None
4,つ,つ,接尾辞,名詞的,*,ツ
5,の,の,助詞,格助詞,*,ノ
6,大島,大島,名詞,普通名詞,*,オオシマ
7,が,が,助詞,格助詞,*,ガ
8,ある,有る,動詞,非自立可能,五段-ラ行,アル
9,。,。,補助記号,句点,*,*


In [14]:
dict_table = data[data["pos1"].isin("名詞、代名詞、形状詞、形容詞、連体詞".split("、"))][attr].drop_duplicates(subset="lemma").reset_index(drop=True)

In [15]:
from pykakasi import kakasi

In [16]:
kks = kakasi()
kks.setMode("K", "H")
conv = kks.getConverter()
conv.do("カタカナ")

'かたかな'

In [17]:
def furiganize(text):
    kks = kakasi()
    kks.setMode("K", "H")
    conv = kks.getConverter()
    
    tagger = fugashi.Tagger()
    
    def convert(toktext):
        from extra import stop_words
        
        token, text = toktext
        feature = token.feature
        
        if feature.orthBase is None:
            out = str(token)
        elif (feature.orthBase, conv.do(feature.kanaBase)) in stop_words:
            out = feature.orth        
        elif feature.kana == "*" or feature.kana == feature.orth:
            out = feature.orth
        else:
            kana = conv.do(feature.kana)
            
            if kana == feature.orth:
                out = feature.orth
            else:
                k = -1
                
                while feature.orth[k] == kana[k]:
                    k -= 1
                    
                k += 1
                
                if k == 0:
                    out = rf"\f{{{feature.orth}}}{{{kana}}}"
                else:                   
                    out = rf"\f{{{feature.orth[:k]}}}{{{kana[:k]}}}{feature.orth[k:]}"
        
        return token, out
            
    def box(toktext):
        token, text = toktext
        return token, rf"\p{{{text}}}"
    
    def wrap(token):
        return token, ""
    
    def finish(toktext):
        token, text = toktext
        return text
    
    toktexts = [convert(wrap(token)) for token in tagger(text)]
    
    prev_token, prev_text = None, None
    box = False
    last = len(toktexts) - 1
    
    def should_close_box(token):
        return (token.feature.pos1 in ["名詞", "代名詞", "形状詞", "連体詞", "副詞", "接続詞", "感動詞", "動詞", "形容詞", "接頭辞"]
                or should_keep_box(token))
    
    def should_keep_box(token):
        return token.feature.pos1 == "補助記号" and token.feature.orth in ["「", "（", "＜", "(", "["]
    
    keep_box = False
    
    for k, (token, text) in enumerate(toktexts):
        if k == 0:
            text = r"\p{" + text
        elif should_close_box(token) and not keep_box:
            text = r"}\p{" + text
            
        keep_box = should_keep_box(token)
            
        if k == last:
            text = text + r"}"
            
        toktexts[k] = token, text
        prev_token, prev_text = token, text
        
    return str.join(" ", [finish(toktext) for toktext in toktexts])

In [24]:
text = """
次の被害者を待つ女
細い山道で人家はほとんど無く、灯りといえば自販機のみ。友人の家はそんな場所に建っていた。遊びに行ったとき、深夜まで話し込んでしまい、気が付くと23時を回っていた。友人のすすめもあって泊まらせてもらうことにした。

風呂に入り、外を眺める。と、友人宅のそばにぽつんとある自販機の横に女の人が立っていた。 こんな夜中に、と不思議に思ったが、深くは気にせず眠りについた。

深夜2時頃、ふと目が覚めトイレに行く。布団に戻る途中で、窓の外の自販機に目がいった。なんとそこには、さっきの女の人がまだ立っていたのだ。気味が悪いと思ったが、もう見なかった事にして布団に入り眠りについた。

翌朝、昨夜の出来事を友人に話した。すると、「昔あの場所で事故があってね」と顔色変えずに話しだす友人。「見える人は見えるし、見えない人は夜中にそこでジュース買ってるよ」

どうも、友人は「見えない人」でらしい。さすがに夜そこでジュースを買うことはしないが。霊感のある知人は、「あの女の人は次の被害者になる人をじっと待っている」と言っていたらしい。近寄らない方がいいとも言われたみたい。
"""

In [25]:
print(*[furiganize(t) for t in text.split("\n\n")], sep="\n\n")

\p{\f{次}{つぎ} の }\p{\f{被害}{ひがい} \f{者}{しゃ} を }\p{\f{待}{ま}つ }\p{\f{女}{おんな} }\p{\f{細}{ほそ}い }\p{\f{山道}{やまみち} で }\p{\f{人家}{じんか} は }\p{ほとんど }\p{\f{無}{な}く 、 }\p{\f{灯}{あか}り と }\p{いえ ば }\p{\f{自販}{じはん} }\p{\f{機}{き} のみ 。 }\p{\f{友人}{ゆうじん} の }\p{\f{家}{いえ} は }\p{そんな }\p{\f{場所}{ばしょ} に }\p{\f{建}{た}っ て }\p{い た 。 }\p{\f{遊}{あそ}び に }\p{\f{行}{い}っ た }\p{とき 、 }\p{\f{深夜}{しんや} まで }\p{\f{話し込}{はなしこ}ん で }\p{しまい 、 }\p{\f{気}{き} が }\p{\f{付}{つ}く と }\p{23 }\p{\f{時}{じ} を }\p{\f{回}{まわ}っ て }\p{い た 。 }\p{\f{友人}{ゆうじん} の }\p{すすめ も }\p{あっ て }\p{\f{泊}{と}まら せ て }\p{もらう }\p{こと に }\p{し た 。}

\p{\f{風呂}{ふろ} に }\p{入り 、 }\p{外 を }\p{\f{眺}{なが}める 。 と 、 }\p{\f{友人}{ゆうじん} }\p{\f{宅}{たく} の }\p{そば に }\p{ぽつん と }\p{ある }\p{\f{自販}{じはん} }\p{\f{機}{き} の }\p{\f{横}{よこ} に }\p{\f{女}{おんな} の }\p{\f{人}{ひと} が }\p{\f{立}{た}っ て }\p{い た 。 }\p{こんな }\p{\f{夜中}{よなか} に 、 と }\p{\f{不思議}{ふしぎ} に }\p{\f{思}{おも}っ た が 、 }\p{\f{深}{ふか}く は }\p{\f{気}{き} に }\p{せ ず }\p{\f{眠}{ねむ}り に }\p{つい た 。}

\p{\f{深夜}{しんや} }\p{2 }\p{\f{時}{じ} }\p{\f{頃}{ころ} 、 }\p{ふと }\p{\f{目}{め} が }\p{\f{覚}{さ}め 

In [39]:
text2 = r"""
語り手は誰だと思いますか？
壁の穴の中の住人はいくつですか？
語り手は何をする積りですか？
語り手の家の掟は何ですか？
いつ今の穴に引っ越してきたのですか？
前の穴はどんな穴でしたか？
""".strip()

text3= r"""
武君の人生は長い失敗の______だった。
花子ちゃんはヒマラヤで新しい石の一種を______。
私は彼がマフィアに______ことを聞いた。
余り______に頼ってはいけない。
______牢獄にいる武君を見に行かない？
______で木を登っている子供がたくさんいる。
老人は我々にその愉快な殺人______について話した。
花子ちゃんと武君について色々な噂が______。
佐藤さんの犬の______を知りませんか？
これは______の質問ではありません。
凜ちゃんの______のおかげで痴漢は捕まりました。
""".strip()

In [40]:
print(*[r"\item " + furiganize(row) for row in text2.split("\n")], sep="\n")

\item \p{\f{語り手}{かたりて}は}\p{\f{誰}{だれ}だと}\p{\f{思}{おも}いますか？}
\item \p{\f{壁}{かべ}の}\p{\f{穴}{あな}の}\p{中の}\p{\f{住人}{じゅうにん}は}\p{いくつですか？}
\item \p{\f{語り手}{かたりて}は}\p{何を}\p{する}\p{\f{積}{つも}りですか？}
\item \p{\f{語り手}{かたりて}の}\p{\f{家}{いえ}の}\p{\f{掟}{おきて}は}\p{\f{何}{なん}ですか？}
\item \p{いつ}\p{今の}\p{\f{穴}{あな}に}\p{\f{引っ越}{ひっこ}して}\p{きたのですか？}
\item \p{前の}\p{\f{穴}{あな}は}\p{どんな}\p{\f{穴}{あな}でしたか？}


In [17]:
pd.concat([pd.DataFrame.from_dict({"word": str(token), **token.feature._asdict()}, orient="index").T
           for token in tagger(text3)]).reset_index(drop=True)[attr]

,orth,lemma,pos1,cType,kana
0,武,タケシ,名詞,*,タケシ
1,君,君,接尾辞,*,クン
2,の,の,助詞,*,ノ
3,人生,人生,名詞,*,ジンセイ
4,は,は,助詞,*,ハ
5,長い,長い,形容詞,形容詞,ナガイ
6,失敗,失敗,名詞,*,シッパイ
7,の,の,助詞,*,ノ
8,_,＿,補助記号,*,*
9,_,＿,補助記号,*,*


In [18]:
from extra import stop_words
stop_words

[('車', 'くるま'),
 ('上', 'うえ'),
 ('下', 'した'),
 ('入る', 'はいる'),
 ('火', 'ひ'),
 ('外', 'そと'),
 ('出る', 'でる'),
 ('出す', 'だす'),
 ('大きい', 'おおきい'),
 ('大きな', 'おおきな'),
 ('山', 'やま'),
 ('中', 'なか'),
 ('何', 'なに'),
 ('時', 'とき'),
 ('見る', 'みる'),
 ('今', 'いま'),
 ('前', 'まえ'),
 ('話す', 'はなす'),
 ('一', 'いち'),
 ('二', 'に'),
 ('三', 'さん'),
 ('四', 'よん'),
 ('五', 'ご'),
 ('六', 'ろく'),
 ('七', 'しち'),
 ('八', 'はち'),
 ('九', 'きゅう'),
 ('十', 'じゅう'),
 ('百', 'ひゃく'),
 ('千', 'せん'),
 ('万', 'まん')]

In [19]:
tagger(text)[0].feature.pos1

'名詞'

In [20]:
import jamdict
jmd = jamdict.Jamdict()

ModuleNotFoundError: No module named 'jamdict'

In [21]:
result = jmd.lookup("や")

NameError: name 'jmd' is not defined

In [23]:
result.entries[5].senses[0].pos[0]

NameError: name 'result' is not defined

In [24]:
tokens[8].feature.pos1

'助詞'

In [25]:
[entry for entry in result.entries if any(["particle" in sense.pos for sense in entry.senses])]

NameError: name 'result' is not defined

In [26]:
pd.concat([pd.DataFrame.from_dict({"word": str(token), **token.feature._asdict()}, orient="index").T
           for token in tagger("健二郎さんは今夜お酒ばかりを飲んでいます。")]).reset_index(drop=True)[["orth", "lemma", "pos1", "pos2"]]

,orth,lemma,pos1,pos2
0,健二郎,ケンジロウ,名詞,固有名詞
1,さん,さん,接尾辞,名詞的
2,は,は,助詞,係助詞
3,今夜,今夜,名詞,普通名詞
4,お,御,接頭辞,*
5,酒,酒,名詞,普通名詞
6,ばかり,ばかり,助詞,副助詞
7,を,を,助詞,格助詞
8,飲ん,飲む,動詞,一般
9,で,て,助詞,接続助詞


In [27]:
pd.concat([pd.DataFrame.from_dict({"word": str(token), **token.feature._asdict()}, orient="index").T
           for token in tagger("誰かがいる。")]).reset_index(drop=True)[["orth", "lemma", "pos1", "pos2"]]

,orth,lemma,pos1,pos2
0,誰,誰,代名詞,*
1,か,か,助詞,副助詞
2,が,が,助詞,格助詞
3,いる,居る,動詞,非自立可能
4,。,。,補助記号,句点
